# 필요 라이브러리 Import

In [ ]:
import os
import json

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import glob

# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/빅데이터및 AI

In [ ]:
ls

## 주석, 이미지 압축 파일 해제

In [ ]:
!unzip Annotations.zip -d Annotations
!zip -FFv Images.zip --out Images_.zip
!unzip Images_.zip -d Images

In [ ]:
ls

## 주석, 이미지 파일 확인

In [ ]:

img_path = "/content/drive/MyDrive/deeplearning/Mainroad_P01_image"
image = glob.glob(img_path + "/*.jpg")
len(image)


In [ ]:
json_path = "/content/drive/MyDrive/deeplearning/5.Mainroad_P01_json"

In [ ]:
# 주석 파일 오픈
with open(f'Annotations/{annots[0]}') as f:
  json_file = json.load(f)

In [ ]:
# 주석 파일은 Dictionary 타입으로, key와 value로 이루어짐
# key 값 확인
json_file.keys()

In [ ]:
# 'info' value 확인
json_file['info']

In [ ]:
# 'images' value 확인
json_file['images']

In [ ]:
# 'annotations' value 확인
json_file['annotations']

In [ ]:
# 'categories' value 확인
json_file['categories']

In [ ]:
# 이미지 파일 오픈
from PIL import Image
Image.open(f'Images/{images[0]}')

### 이미지에 주석 적용하여 객체 bounding box 표시하기

- PIL 라이브러리의 ImageDraw 모듈 활용
- json의 'annotations' value 이용

- 정답 예시

![image](https://raw.githubusercontent.com/pyeon9/Lecture-BigData-AI/main/image_annotation.png?token=AL7QGG622SZFU32VNRUB4F3BSHPMA)


In [ ]:
# 이미지에 주석 적용
from PIL import ImageDraw

img_open = Image.open(f'Images/{images[0]}').convert('RGBA')
with open(f'Annotations/{annots[0]}') as f:
  json_file = json.load(f)

jf = json_file['annotations']
for i in range(len(jf)):
  draw = ImageDraw.Draw(img_open)
  draw.rectangle((jf[i]['bbox'][0],jf[i]['bbox'][1],jf[i]['bbox'][2]+jf[i]['bbox'][0],jf[i]['bbox'][3]+jf[i]['bbox'][1]), outline=(0,255,0), width = 3)

plt.figure(figsize=(20,10))
plt.imshow(np.array(img_open))
plt.show()

# 카테고리에 따라 파일 분류

- 7 : Stones on road
- 8 : Pothole on road
- 9 : Filled pothole
- 한 이미지에 [7,8,9] 중 2개 이상이 해당하는 경우는 제외
- ex) 한 이미지에 카테고리가 [1, 1, 8] > 8로 분류
- ex) 한 이미지에 카테고리가 [1, 7, 8] > 제외   
</br>
- 주석 파일을 카테고리에 맞는 리스트에 append()

In [ ]:
stone = []
pothole = []
filled = []

for a in annots:
  with open(f'Annotations/{a}') as f:
    json_file = json.load(f)

    jf = json_file['annotations']

    count = [0,0,0]
    tmp = []

    for i in range(len(jf)):
      if jf[i]['category_id'] == 7:
        count[0] += 1
        tmp = a
      elif jf[i]['category_id'] == 8:
        count[1] += 1
        tmp = a
      elif jf[i]['category_id'] == 9:
        count[2] += 1
        tmp = a

    if count[0]+count[1] == 0 or count[0]+count[2] == 0 or count[1]+count[2] == 0:
      if count[0] > 0:
        stone.append(tmp)
      elif count[1] > 0:
        pothole.append(tmp)
      elif count[2] > 0:
        filled.append(tmp)

In [ ]:
print(len(stone), len(pothole), len(filled))

## 분류된 이미지 새 디렉토리로 복사
- shutil 라이브러리의 copy() 함수 활용
- `make_output_dir()` 함수로 디렉토리 구조를 생성
- 주석 파일과 이미지 파일을 각각 "Annotations/"와 "Images/" 디렉토리로 복사

In [ ]:
# 주어진 디렉토리 아래 "Annotations/"와 "Images/"의 두 디렉토리 생성
def make_output_dir(category):
    os.makedirs(f'{category}/Annotations')
    os.makedirs(f'{category}/Images')

In [ ]:
# 카테고리별 디렉토리 생성
make_output_dir('0. stone')
make_output_dir('1. pothole')
make_output_dir('2. filled')

In [ ]:
# 위에서 생성한 디렉토리에 주석 파일과 이미지 파일 이동
import shutil
def copy_files(json_lst, category):
    for j in json_lst:
      from_file_path = os.path.join("Annotations", j)
      to_file_path = os.path.join(category, "Annotations")
      shutil.copy(from_file_path, to_file_path)

      j = j.replace("_BBOX.json", ".png")
      from_file_path = os.path.join("Images", j)
      to_file_path = os.path.join(category, "Images")
      shutil.copy(from_file_path, to_file_path)


In [ ]:
copy_files(stone, '0. stone')
copy_files(pothole, '1. pothole')
copy_files(filled, '2. filled')

In [ ]:
num_stone = os.listdir('0. stone/Images')
num_pothole = os.listdir('1. pothole/Images')
num_filled  = os.listdir('2. filled/Images')

print(len(num_stone), len(num_pothole), len(num_filled))